In [1]:
import os, time
import math
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import imageio
import xml.etree.ElementTree as ET
from tqdm import tqdm_notebook as tqdm

In [38]:
epochs = 4000
batch_size = 64
latent_dim = 100
img_size = 64

In [20]:
cuda = True if torch.cuda.is_available() else False

In [21]:
if not os.path.isdir('sample_images'):
    os.mkdir('sample_images')

In [5]:
# code from https://www.kaggle.com/guillaumedesforges/usable-complete-data-loading-utility
class FullCroppedDogsFolderDataset(torchvision.datasets.vision.VisionDataset):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root, transform=transform, target_transform=target_transform)
        self.transform = transform
        self.target_transform = target_transform
        
        self.samples = self._load_subfolders_images(self.root)
        if len(self.samples) == 0:
            raise RuntimeError("Found 0 files in subfolders of: {}".format(self.root))
            
    def _load_subfolders_images(self, root):
        IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

        def is_valid_file(x):
            return torchvision.datasets.folder.has_file_allowed_extension(x, IMG_EXTENSIONS)

        required_transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize(64),
            torchvision.transforms.CenterCrop(64),
        ])

        imgs = []

        paths = []
        for root, _, fnames in sorted(os.walk(root)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                paths.append(path)

        pbar = tqdm(paths, desc='Loading cropped images')

        for path in pbar:
            if is_valid_file(path):
                # Load image
                img = torchvision.datasets.folder.default_loader(path)

                # Get bounding boxes
                annotation_basename = os.path.splitext(os.path.basename(path))[0]
                annotation_dirname = next(dirname for dirname in os.listdir('data/Annotation/Annotation/') if dirname.startswith(annotation_basename.split('_')[0]))
                annotation_filename = os.path.join('data/Annotation/Annotation', annotation_dirname, annotation_basename)
                tree = ET.parse(annotation_filename)
                root = tree.getroot()
                objects = root.findall('object')
                for o in objects:
                    bndbox = o.find('bndbox')
                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)

                    bbox = (xmin, ymin, xmax, ymax)

                    object_img = required_transforms(img.crop(bbox))
                    imgs.append(object_img)
                
                pbar.set_postfix_str("{} cropped images loaded".format(len(imgs)))

        return imgs
    
    def __getitem__(self, index):
        sample = self.samples[index]
        target = 1
        
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    def __len__(self):
        return len(self.samples)
    
dataset = FullCroppedDogsFolderDataset(
    'data/all-dogs/',
    transform=torchvision.transforms.Compose([
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
    ])
)

/home/sauban/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [22]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [23]:
class Generator(nn.Module):
    # initializers
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = img_size // 16
        self.rel = nn.LeakyReLU(0.2, inplace=True)
        self.l1 = nn.Sequential(nn.Linear(100, 128 * self.init_size ** 2))
        self.deconv1 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.deconv1_bn = nn.BatchNorm2d(64)
        self.deconv2 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(32)
        self.deconv3 = nn.ConvTranspose2d(32, 16, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(16)
        self.deconv4 = nn.ConvTranspose2d(16, 3, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        # x = F.relu(self.deconv1(input))
        x = self.l1(input)
        x = x.view(x.shape[0], 128, self.init_size, self.init_size)
        x = self.rel(self.deconv1_bn(self.deconv1(x)))
        x = self.rel(self.deconv2_bn(self.deconv2(x)))
        x = self.rel(self.deconv3_bn(self.deconv3(x)))
        x = F.tanh(self.deconv4(x))

        return x

In [24]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.convs = nn.Sequential(
            *discriminator_block(3, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        ds_size = img_size // 2 ** 4
        self.fc_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        x = self.convs(img)
        x = x.view(x.shape[0], -1)
        x = self.fc_layer(x)

        return x

In [25]:
adversarial_loss = torch.nn.BCELoss()

generator = Generator()
discriminator = Discriminator()

In [26]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [27]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (convs): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc_layer

In [28]:
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)

In [29]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [30]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [31]:
len(dataset)

22125

In [39]:
print('training start!')
start_time = time.time()
for epoch in range(2000,epochs):
    for i, (imgs, _) in enumerate(data_loader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))
        #  Train Generator
        
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)
        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        
        optimizer_D.zero_grad()

        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        if i % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, epochs, i, len(data_loader), d_loss.item(), g_loss.item())
            )
        batches_done = epoch * len(data_loader) + i
    save_image(gen_imgs.data[:16], "sample_images/%d.png" % epoch, nrow=4, normalize=True)
end_time = time.time()
total_time = end_time - start_time
total_time

training start!
[Epoch 2000/4000] [Batch 0/345] [D loss: 0.187808] [G loss: 4.277997]
[Epoch 2000/4000] [Batch 100/345] [D loss: 0.034388] [G loss: 0.922541]
[Epoch 2000/4000] [Batch 200/345] [D loss: 0.250058] [G loss: 3.250356]
[Epoch 2000/4000] [Batch 300/345] [D loss: 0.848787] [G loss: 0.534458]
[Epoch 2001/4000] [Batch 0/345] [D loss: 0.016838] [G loss: 2.581580]
[Epoch 2001/4000] [Batch 100/345] [D loss: 0.700082] [G loss: 2.042100]
[Epoch 2001/4000] [Batch 200/345] [D loss: 0.088158] [G loss: 1.897862]
[Epoch 2001/4000] [Batch 300/345] [D loss: 0.271652] [G loss: 2.906990]
[Epoch 2002/4000] [Batch 0/345] [D loss: 0.057162] [G loss: 4.082058]
[Epoch 2002/4000] [Batch 100/345] [D loss: 0.665865] [G loss: 2.237541]
[Epoch 2002/4000] [Batch 200/345] [D loss: 0.226341] [G loss: 0.921852]
[Epoch 2002/4000] [Batch 300/345] [D loss: 0.119954] [G loss: 1.923040]
[Epoch 2003/4000] [Batch 0/345] [D loss: 0.103300] [G loss: 2.597870]
[Epoch 2003/4000] [Batch 100/345] [D loss: 0.946894] [G 

[Epoch 2028/4000] [Batch 300/345] [D loss: 0.076765] [G loss: 5.138154]
[Epoch 2029/4000] [Batch 0/345] [D loss: 0.069021] [G loss: 4.173796]
[Epoch 2029/4000] [Batch 100/345] [D loss: 0.082861] [G loss: 3.784750]
[Epoch 2029/4000] [Batch 200/345] [D loss: 0.310949] [G loss: 2.194400]
[Epoch 2029/4000] [Batch 300/345] [D loss: 0.179843] [G loss: 2.505526]
[Epoch 2030/4000] [Batch 0/345] [D loss: 0.237976] [G loss: 2.594803]
[Epoch 2030/4000] [Batch 100/345] [D loss: 0.470089] [G loss: 4.129476]
[Epoch 2030/4000] [Batch 200/345] [D loss: 0.546146] [G loss: 2.299675]
[Epoch 2030/4000] [Batch 300/345] [D loss: 0.190345] [G loss: 2.497882]
[Epoch 2031/4000] [Batch 0/345] [D loss: 0.057700] [G loss: 0.816535]
[Epoch 2031/4000] [Batch 100/345] [D loss: 0.338136] [G loss: 3.954540]
[Epoch 2031/4000] [Batch 200/345] [D loss: 0.068386] [G loss: 0.985329]
[Epoch 2031/4000] [Batch 300/345] [D loss: 0.068881] [G loss: 1.744641]
[Epoch 2032/4000] [Batch 0/345] [D loss: 0.189430] [G loss: 1.559491]


[Epoch 2057/4000] [Batch 200/345] [D loss: 0.055310] [G loss: 2.066029]
[Epoch 2057/4000] [Batch 300/345] [D loss: 0.367289] [G loss: 1.353070]
[Epoch 2058/4000] [Batch 0/345] [D loss: 0.328297] [G loss: 0.569761]
[Epoch 2058/4000] [Batch 100/345] [D loss: 0.701591] [G loss: 0.408002]
[Epoch 2058/4000] [Batch 200/345] [D loss: 0.141496] [G loss: 3.808513]
[Epoch 2058/4000] [Batch 300/345] [D loss: 0.549335] [G loss: 2.625046]
[Epoch 2059/4000] [Batch 0/345] [D loss: 0.305462] [G loss: 2.085852]
[Epoch 2059/4000] [Batch 100/345] [D loss: 0.669073] [G loss: 2.863592]
[Epoch 2059/4000] [Batch 200/345] [D loss: 0.234945] [G loss: 2.501104]
[Epoch 2059/4000] [Batch 300/345] [D loss: 0.541855] [G loss: 4.704123]
[Epoch 2060/4000] [Batch 0/345] [D loss: 0.329670] [G loss: 3.492132]
[Epoch 2060/4000] [Batch 100/345] [D loss: 0.179288] [G loss: 1.489595]
[Epoch 2060/4000] [Batch 200/345] [D loss: 1.160684] [G loss: 5.012121]
[Epoch 2060/4000] [Batch 300/345] [D loss: 0.051328] [G loss: 0.990987

[Epoch 2086/4000] [Batch 100/345] [D loss: 0.711226] [G loss: 2.374531]
[Epoch 2086/4000] [Batch 200/345] [D loss: 0.111409] [G loss: 2.141356]
[Epoch 2086/4000] [Batch 300/345] [D loss: 0.276962] [G loss: 4.077724]
[Epoch 2087/4000] [Batch 0/345] [D loss: 0.359228] [G loss: 3.301032]
[Epoch 2087/4000] [Batch 100/345] [D loss: 0.713130] [G loss: 4.191025]
[Epoch 2087/4000] [Batch 200/345] [D loss: 0.402313] [G loss: 4.549834]
[Epoch 2087/4000] [Batch 300/345] [D loss: 0.266447] [G loss: 0.751129]
[Epoch 2088/4000] [Batch 0/345] [D loss: 0.541467] [G loss: 2.054497]
[Epoch 2088/4000] [Batch 100/345] [D loss: 0.128173] [G loss: 0.722636]
[Epoch 2088/4000] [Batch 200/345] [D loss: 0.101877] [G loss: 2.253592]
[Epoch 2088/4000] [Batch 300/345] [D loss: 0.281964] [G loss: 2.509735]
[Epoch 2089/4000] [Batch 0/345] [D loss: 0.141161] [G loss: 2.635070]
[Epoch 2089/4000] [Batch 100/345] [D loss: 0.641188] [G loss: 1.357850]
[Epoch 2089/4000] [Batch 200/345] [D loss: 0.155281] [G loss: 0.545210

[Epoch 2115/4000] [Batch 0/345] [D loss: 0.136803] [G loss: 2.060571]
[Epoch 2115/4000] [Batch 100/345] [D loss: 1.257712] [G loss: 1.100617]
[Epoch 2115/4000] [Batch 200/345] [D loss: 0.156364] [G loss: 0.372180]
[Epoch 2115/4000] [Batch 300/345] [D loss: 0.247362] [G loss: 2.655355]
[Epoch 2116/4000] [Batch 0/345] [D loss: 0.791683] [G loss: 1.702582]
[Epoch 2116/4000] [Batch 100/345] [D loss: 0.076628] [G loss: 2.664429]
[Epoch 2116/4000] [Batch 200/345] [D loss: 0.312951] [G loss: 1.712010]
[Epoch 2116/4000] [Batch 300/345] [D loss: 0.221048] [G loss: 3.773384]
[Epoch 2117/4000] [Batch 0/345] [D loss: 0.095363] [G loss: 2.404982]
[Epoch 2117/4000] [Batch 100/345] [D loss: 0.357327] [G loss: 2.130802]
[Epoch 2117/4000] [Batch 200/345] [D loss: 0.657674] [G loss: 3.032858]
[Epoch 2117/4000] [Batch 300/345] [D loss: 0.319491] [G loss: 2.507924]
[Epoch 2118/4000] [Batch 0/345] [D loss: 0.394156] [G loss: 4.981941]
[Epoch 2118/4000] [Batch 100/345] [D loss: 0.183497] [G loss: 0.515790]


...

[Epoch 3897/4000] [Batch 200/345] [D loss: 0.233446] [G loss: 1.905580]
[Epoch 3897/4000] [Batch 300/345] [D loss: 0.092066] [G loss: 4.377278]
[Epoch 3898/4000] [Batch 0/345] [D loss: 0.757780] [G loss: 1.840751]
[Epoch 3898/4000] [Batch 100/345] [D loss: 0.684314] [G loss: 0.605194]
[Epoch 3898/4000] [Batch 200/345] [D loss: 0.239941] [G loss: 2.828299]
[Epoch 3898/4000] [Batch 300/345] [D loss: 0.137705] [G loss: 3.311958]
[Epoch 3899/4000] [Batch 0/345] [D loss: 0.443253] [G loss: 1.118037]
[Epoch 3899/4000] [Batch 100/345] [D loss: 0.086524] [G loss: 5.397116]
[Epoch 3899/4000] [Batch 200/345] [D loss: 0.074472] [G loss: 1.542407]
[Epoch 3899/4000] [Batch 300/345] [D loss: 0.217231] [G loss: 2.728686]
[Epoch 3900/4000] [Batch 0/345] [D loss: 0.509859] [G loss: 3.315808]
[Epoch 3900/4000] [Batch 100/345] [D loss: 0.312675] [G loss: 5.205719]
[Epoch 3900/4000] [Batch 200/345] [D loss: 0.949673] [G loss: 3.877412]
[Epoch 3900/4000] [Batch 300/345] [D loss: 0.469755] [G loss: 1.238450

[Epoch 3926/4000] [Batch 100/345] [D loss: 0.338568] [G loss: 3.769298]
[Epoch 3926/4000] [Batch 200/345] [D loss: 0.259800] [G loss: 2.813929]
[Epoch 3926/4000] [Batch 300/345] [D loss: 0.281598] [G loss: 4.176448]
[Epoch 3927/4000] [Batch 0/345] [D loss: 0.250819] [G loss: 2.158740]
[Epoch 3927/4000] [Batch 100/345] [D loss: 0.397140] [G loss: 5.799450]
[Epoch 3927/4000] [Batch 200/345] [D loss: 0.221101] [G loss: 1.128527]
[Epoch 3927/4000] [Batch 300/345] [D loss: 0.066928] [G loss: 2.532880]
[Epoch 3928/4000] [Batch 0/345] [D loss: 0.605589] [G loss: 1.109798]
[Epoch 3928/4000] [Batch 100/345] [D loss: 0.168774] [G loss: 2.888771]
[Epoch 3928/4000] [Batch 200/345] [D loss: 0.086085] [G loss: 4.319204]
[Epoch 3928/4000] [Batch 300/345] [D loss: 0.193563] [G loss: 2.988847]
[Epoch 3929/4000] [Batch 0/345] [D loss: 0.250470] [G loss: 0.657552]
[Epoch 3929/4000] [Batch 100/345] [D loss: 0.265396] [G loss: 2.537466]
[Epoch 3929/4000] [Batch 200/345] [D loss: 0.074307] [G loss: 3.496606

[Epoch 3955/4000] [Batch 0/345] [D loss: 0.592599] [G loss: 0.945044]
[Epoch 3955/4000] [Batch 100/345] [D loss: 0.856630] [G loss: 2.347768]
[Epoch 3955/4000] [Batch 200/345] [D loss: 0.269762] [G loss: 2.656681]
[Epoch 3955/4000] [Batch 300/345] [D loss: 0.475908] [G loss: 3.720196]
[Epoch 3956/4000] [Batch 0/345] [D loss: 0.181107] [G loss: 1.061802]
[Epoch 3956/4000] [Batch 100/345] [D loss: 0.019391] [G loss: 4.101835]
[Epoch 3956/4000] [Batch 200/345] [D loss: 0.168028] [G loss: 4.322848]
[Epoch 3956/4000] [Batch 300/345] [D loss: 0.645354] [G loss: 3.164439]
[Epoch 3957/4000] [Batch 0/345] [D loss: 0.132055] [G loss: 2.368202]
[Epoch 3957/4000] [Batch 100/345] [D loss: 1.034093] [G loss: 5.742636]
[Epoch 3957/4000] [Batch 200/345] [D loss: 0.702057] [G loss: 7.579775]
[Epoch 3957/4000] [Batch 300/345] [D loss: 0.309807] [G loss: 2.600195]
[Epoch 3958/4000] [Batch 0/345] [D loss: 0.019830] [G loss: 2.319583]
[Epoch 3958/4000] [Batch 100/345] [D loss: 1.105768] [G loss: 4.367462]


[Epoch 3983/4000] [Batch 300/345] [D loss: 1.538944] [G loss: 4.102169]
[Epoch 3984/4000] [Batch 0/345] [D loss: 0.100792] [G loss: 4.234737]
[Epoch 3984/4000] [Batch 100/345] [D loss: 0.557207] [G loss: 2.063639]
[Epoch 3984/4000] [Batch 200/345] [D loss: 0.023561] [G loss: 1.354961]
[Epoch 3984/4000] [Batch 300/345] [D loss: 0.187048] [G loss: 2.292398]
[Epoch 3985/4000] [Batch 0/345] [D loss: 0.079133] [G loss: 1.735288]
[Epoch 3985/4000] [Batch 100/345] [D loss: 0.053050] [G loss: 2.138347]
[Epoch 3985/4000] [Batch 200/345] [D loss: 0.169822] [G loss: 2.716680]
[Epoch 3985/4000] [Batch 300/345] [D loss: 0.084310] [G loss: 3.717323]
[Epoch 3986/4000] [Batch 0/345] [D loss: 0.128104] [G loss: 2.882123]
[Epoch 3986/4000] [Batch 100/345] [D loss: 0.393237] [G loss: 0.997451]
[Epoch 3986/4000] [Batch 200/345] [D loss: 0.160634] [G loss: 4.288697]
[Epoch 3986/4000] [Batch 300/345] [D loss: 0.113233] [G loss: 2.356214]
[Epoch 3987/4000] [Batch 0/345] [D loss: 0.127767] [G loss: 0.701023]


17615.090319633484

In [16]:
epochs

15

In [18]:
images = []
for epoch in range(epochs):
    img_name = 'mnist-results/' + str(epoch) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave('mnist-results/animation.gif', images, fps=5)